In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
def get_basic_info(key):
    key1 = key.split("/")[0]
    if "ARCTIC" in key1:
        data_type = "Ground Truth"
        speaker = key.split("/")[1]
        accent = spk2acc[speaker]
    elif "PD-AST"==key1:
        data_type = "SPAccenT"
        speaker, accent = key.split("/")[1:]
    else:
        data_type = "AC_"
        if "smaller" in key1:
            data_type += "small"
        elif "small" in key1:
            data_type += "normal"
        else:
            data_type += "large"
        if "VCTK" in key1:
            data_type += "_VCTK"
        speaker = "SLT"
        accent = mode
    return data_type, speaker, accent

spk2acc = {
    "SLT": "English",
    "ASI": "Hindi",
    "TNI": "Hindi",
    "HKK": "Korean",
    "YDCK": "Korean",
}

In [3]:
mode = "Hindi"
if mode=="Hindi":
    labels = [
        "CMU-ARCTIC/SLT",
        "L2-ARCTIC/ASI",
        "L2-ARCTIC/TNI",
        "PD-AST/SLT/English",
        "PD-AST/SLT/Hindi",
        "PD-AST/ASI/Hindi",
        "PD-AST/TNI/Hindi",
        # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
        # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
        # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
        # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
        
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    ]
elif mode=="Korean":
    labels = [
        "L2-ARCTIC/HKK",
        "L2-ARCTIC/YDCK",
        "PD-AST/SLT/English",
        "PD-AST/SLT/Korean",
        "PD-AST/HKK/Korean",
        "PD-AST/YDCK/Korean",
    ]

In [4]:
save_dir = f"evaluation/wers/individual/"

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("WER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)

basics = []
wers = []
cers = []
for dirname in labels:
    data_type, speaker, accent = get_basic_info(dirname)
    path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
    wer_list, cer_list = np.load(path)
    wers += [list(wer_list)]
    cers += [list(cer_list)]
    basics += [[data_type, speaker, accent]]
    
new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("WER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)
array = np.concatenate([basics, wers], axis=1)
werdf = pd.DataFrame(array, columns=new_columns)
werdf.loc[:, "WER"] =  werdf.loc[:, "WER"].values.astype(float)

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("CER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)
array = np.concatenate([basics, cers], axis=1)
cerdf = pd.DataFrame(array, columns=new_columns)
cerdf.loc[:, "CER"] =  cerdf.loc[:, "CER"].values.astype(float)
werdf

basics                        WER                                \
         Data Type Speaker   Accent      tiny      base     small    medium   
0     Ground Truth     SLT  English  0.059016  0.033880  0.025137  0.025137   
1     Ground Truth     ASI    Hindi  0.145355  0.093989  0.067760  0.062295   
2     Ground Truth     TNI    Hindi  0.191257  0.125683  0.077596  0.067760   
3         SPAccenT     SLT  English  0.036066  0.026230  0.021858  0.028415   
4         SPAccenT     SLT    Hindi  0.203279  0.149727  0.113661  0.110383   
5         SPAccenT     ASI    Hindi  0.158470  0.123497  0.092896  0.090710   
6         SPAccenT     TNI    Hindi  0.157377  0.104918  0.085246  0.084153   
7        AC_normal     SLT    Hindi  0.275410  0.249180  0.180328  0.157377   
8         AC_large     SLT    Hindi  0.251366  0.228415  0.186885  0.161749   
9   AC_normal_VCTK     SLT    Hindi  0.274317  0.248087  0.183607  0.148634   
10   AC_large_VCTK     SLT    Hindi  0.226230  0.170492  0.140984  0.111475   

              
       large  
0   0.019672  
1   0.046995  
2   0.057923  
3   0.017486  
4   0.085246  
5   0.093989  
6   0.076503  
7   0.128962  
8   0.113661  
9   0.125683  
10  0.075410

In [5]:
save_dir = f"evaluation/accent/"

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["classification score (before softmax)", "classification prob.", "AECS ratio"]:
    new_columns += [("accent", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)

basics = []
scores = []
for dirname in labels:
    data_type, speaker, accent = get_basic_info(dirname)
    path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
    raws, probs, embs = np.load(path)
    scores += [[raws.mean(), probs.mean(), embs.mean()]]
    basics += [[data_type, speaker, accent]]
    
array = np.concatenate([basics, scores], axis=1)
accentdf = pd.DataFrame(array, columns=new_columns)
accentdf.loc[:, "accent"] =  np.round(accentdf.loc[:, "accent"].values.astype(float), 4)
accentdf

basics                                                 accent  \
         Data Type Speaker   Accent classification score (before softmax)   
0     Ground Truth     SLT  English                               -9.2517   
1     Ground Truth     ASI    Hindi                               13.3049   
2     Ground Truth     TNI    Hindi                               17.5397   
3         SPAccenT     SLT  English                               -9.0417   
4         SPAccenT     SLT    Hindi                               13.9551   
5         SPAccenT     ASI    Hindi                               11.8960   
6         SPAccenT     TNI    Hindi                               14.6826   
7        AC_normal     SLT    Hindi                               16.5069   
8         AC_large     SLT    Hindi                               13.1740   
9   AC_normal_VCTK     SLT    Hindi                               17.8086   
10   AC_large_VCTK     SLT    Hindi                               17.1122   

                                    
   classification prob. AECS ratio  
0                0.0000    -0.4621  
1                0.6320     0.2239  
2                0.8838     0.2867  
3                0.0000    -0.5686  
4                0.8193     0.5686  
5                0.3538     0.2373  
6                0.7237     0.2771  
7                0.9435     0.4878  
8                0.8351     0.4362  
9                0.9483     0.5256  
10               0.9069     0.5053

In [6]:
result = pd.concat([werdf[["basics", "WER"]],  cerdf[["CER"]], accentdf[["accent"]]], axis=1)
result

basics                        WER                                \
         Data Type Speaker   Accent      tiny      base     small    medium   
0     Ground Truth     SLT  English  0.059016  0.033880  0.025137  0.025137   
1     Ground Truth     ASI    Hindi  0.145355  0.093989  0.067760  0.062295   
2     Ground Truth     TNI    Hindi  0.191257  0.125683  0.077596  0.067760   
3         SPAccenT     SLT  English  0.036066  0.026230  0.021858  0.028415   
4         SPAccenT     SLT    Hindi  0.203279  0.149727  0.113661  0.110383   
5         SPAccenT     ASI    Hindi  0.158470  0.123497  0.092896  0.090710   
6         SPAccenT     TNI    Hindi  0.157377  0.104918  0.085246  0.084153   
7        AC_normal     SLT    Hindi  0.275410  0.249180  0.180328  0.157377   
8         AC_large     SLT    Hindi  0.251366  0.228415  0.186885  0.161749   
9   AC_normal_VCTK     SLT    Hindi  0.274317  0.248087  0.183607  0.148634   
10   AC_large_VCTK     SLT    Hindi  0.226230  0.170492  0.140984  0.111475   

                   CER                                          \
       large      tiny      base     small    medium     large   
0   0.019672  0.023256  0.011420  0.007475  0.007475  0.005814   
1   0.046995  0.058970  0.037168  0.025125  0.025332  0.016196   
2   0.057923  0.099252  0.060839  0.036752  0.030939  0.027409   
3   0.017486  0.011836  0.007683  0.006437  0.008098  0.004568   
4   0.085246  0.093023  0.062708  0.046719  0.046096  0.036545   
5   0.093989  0.072051  0.055440  0.039037  0.039452  0.043812   
6   0.076503  0.066445  0.045473  0.038206  0.033430  0.032392   
7   0.128962  0.139120  0.129983  0.096553  0.083264  0.072674   
8   0.113661  0.126453  0.117940  0.093646  0.081811  0.059385   
9   0.125683  0.140781  0.119394  0.086379  0.067276  0.056063   
10  0.075410  0.114826  0.082226  0.066445  0.051703  0.034261   

                                  accent                                  
   classification score (before softmax) classification prob. AECS ratio  
0                                -9.2517               0.0000    -0.4621  
1                                13.3049               0.6320     0.2239  
2                                17.5397               0.8838     0.2867  
3                                -9.0417               0.0000    -0.5686  
4                                13.9551               0.8193     0.5686  
5                                11.8960               0.3538     0.2373  
6                                14.6826               0.7237     0.2771  
7                                16.5069               0.9435     0.4878  
8                                13.1740               0.8351     0.4362  
9                                17.8086               0.9483     0.5256  
10                               17.1122               0.9069     0.5053

In [7]:
result = pd.concat([werdf[["basics", "WER"]],  cerdf[["CER"]], accentdf[["accent"]]], axis=1)

In [9]:
import math
def get_latex_from_pandas(meandf, columns, indices, title="Title", label="Label", itvldf=None, multicolumns={}, multiindices={}, midrules=[], pm_list=[], baseround_list=[], comparison=[], highestmin=[], twocolumn=False):
    if len(baseround_list)==0:
        baseround_list = [3]*len(columns)

    indexnum = 1 + int(len(multiindices))
    bestdf = meandf.copy()
    bestdf[:] = 0
    for com in comparison:
        a = meandf.loc[np.array(list(indices.keys()))[com], list(columns.keys())]
        a.iloc[:, highestmin] = -a.iloc[:, highestmin]
        a = a==a.max()
        for key in a.index:
            bestdf.loc[key] = a.loc[key].astype(float)

    if twocolumn:
        print("\\begin{table*}[!h]")
    else:
        print("\\begin{table}[!h]")
    print("\caption{" + title + "}")
    print("\label{table:" + label + "}")
    print("\\centering")
    cl_def = ""
    cl_def += "l"*(indexnum)
    cl_def += "c"*len(columns)

    text = "\\begin{tabular}{" + cl_def + "}"
    print(text)
    print("\\toprule")
    if len(multicolumns)>0:
        print(" & ".join([""]*(indexnum) + [f"\multicolumn{{{multicolumns[key]}}}{{c}}{{{key}}}" for key in multicolumns]) + "\\\\")
        linetxt = ""
        start = indexnum+1
        for key in multicolumns:
            end = start+multicolumns[key]-1
            linetxt += f"\\cmidrule(lr){{{start}-{end}}}"
            start = end+1
        print(linetxt)
    print(f" & ".join([""]*(indexnum+1) + list(columns.values()))[2:] + "\\\\")
    print(f"\midrule")
    for i in range(len(indices)):
        key = list(indices.keys())[i]
        keyname = indices[key]
        start = f"{keyname} &"
        vlist = []
        for k in range(len(columns)):
            base_round = baseround_list[k]
            valuekey = list(columns.keys())[k]
            v = meandf.loc[key, valuekey]
            if str(v)=="nan":
                vstr = "-"
            else:
                if v>0:
                    val = f"{np.round(v,base_round)}".ljust(base_round+2, '0')
                elif v==0:
                    val = "0."+"0"*base_round
                else:
                    val = f"{np.round(v,base_round)}".ljust(base_round+2, '0')
                    pass
                vstr = str(val)
                if bestdf.loc[key, valuekey]==1.0:
                    vstr = f"\\textbf{{{vstr}}}"
            if k in pm_list:
                ivl = itvldf.loc[key, valuekey]
                ivl = f"{np.round(ivl,base_round)}".ljust(base_round+2, '0')
                vstr = vstr + f"{{\\tiny $\\pm$ {{{ivl}}} }}"
            vlist += [vstr]
        print(start + " & ".join(vlist) + "\\\\")
        if i+1 in midrules:
            print("\midrule")
    print("\\bottomrule")
    print("\end{tabular}")
    if twocolumn:
        print("\end{table*}")
    else:
        print("\end{table}")

In [104]:
df = result.iloc[:,3:]
df.index = ["---".join(a) for a in basics]
df = df[[("WER", "large"), ("CER", "large"), ("accent", "classification prob."), ("accent", "AECS ratio")]]
df.columns = ["WER", "CER", "accent prob.", "AECS ratio"]
meandf = pd.concat([df, pd.DataFrame(50*np.ones((len(df), 2)), index=df.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])], axis=1)
itvldf = pd.concat([df, pd.DataFrame(0.1**np.ones((len(df), 2)), index=df.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])], axis=1)

title = "Objective Evaluation"
label = "result_conversion"
columns = {
    "Speech-MUSHRA": "MUSURA ($\\uparrow$)",
    "WER": "WER ($\\downarrow$)",
    "CER": "CER ($\\downarrow$)",
    "Accent-MUSHRA": "MUSURA ($\\uparrow$)",
    "accent prob.": "Accent Probability ($\\uparrow$)",
    "AECS ratio": "AECS Ratio ($\\uparrow$)",
}
indices = {
    "Ground Truth---SLT---English": "Ground Truth (American)",
    "SPAccenT---SLT---English": "SPAccenT (American)",
    "SPAccenT---SLT---Hindi": "SPAccenT (Hindi)",
    "AC_large---SLT---Hindi": "Converted",
    "AC_large_VCTK---SLT---Hindi": "Converted (ours)",
}
multicolumns = { # {}
    "Speech Quality": 3,
    "Accentedness": 3,
}
multiindices = {} # {}
midrules = [3] # []
pm_list = [0, 3] # []
baseround_list = [2, 3, 3, 2, 3, 3]
comparison = [ # []
    [3,4],
]
highestmin = [1, 2] # []
twocolumn = True # False

get_latex_from_pandas(meandf, columns, indices, title, label, itvldf, multicolumns, multiindices, midrules, pm_list, baseround_list, comparison, highestmin, twocolumn)

\begin{table*}[!h]
\caption{Objective Evaluation}
\label{table:result_conversion}
\centering
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{Speech Quality} & \multicolumn{3}{c}{Accentedness}\\
\cmidrule(lr){2-4}\cmidrule(lr){5-7}
  & MUSURA ($\uparrow$) & WER ($\downarrow$) & CER ($\downarrow$) & MUSURA ($\uparrow$) & Accent Probability ($\uparrow$) & AECS Ratio ($\uparrow$)\\
\midrule
Ground Truth (American) &50.0{\tiny $\pm$ {0.10} } & 0.020 & 0.006 & 50.0{\tiny $\pm$ {0.10} } & 0.000 & -0.462\\
SPAccenT (American) &50.0{\tiny $\pm$ {0.10} } & 0.017 & 0.005 & 50.0{\tiny $\pm$ {0.10} } & 0.000 & -0.569\\
SPAccenT (Hindi) &50.0{\tiny $\pm$ {0.10} } & 0.085 & 0.037 & 50.0{\tiny $\pm$ {0.10} } & 0.819 & 0.569\\
\midrule
Converted &\textbf{50.0}{\tiny $\pm$ {0.10} } & 0.114 & 0.059 & \textbf{50.0}{\tiny $\pm$ {0.10} } & 0.835 & 0.436\\
Converted (ours) &\textbf{50.0}{\tiny $\pm$ {0.10} } & \textbf{0.075} & \textbf{0.034} & \textbf{50.0}{\tiny $\pm$ {0.10} } & \textbf{0.907} & \te

In [105]:
meandf

,WER,CER,accent prob.,AECS ratio,Speech-MUSHRA,Accent-MUSHRA
Ground Truth---SLT---English,0.019672,0.005814,0.0000,-0.4621,50.0,50.0
Ground Truth---ASI---Hindi,0.046995,0.016196,0.6320,0.2239,50.0,50.0
Ground Truth---TNI---Hindi,0.057923,0.027409,0.8838,0.2867,50.0,50.0
SPAccenT---SLT---English,0.017486,0.004568,0.0000,-0.5686,50.0,50.0
SPAccenT---SLT---Hindi,0.085246,0.036545,0.8193,0.5686,50.0,50.0
SPAccenT---ASI---Hindi,0.093989,0.043812,0.3538,0.2373,50.0,50.0
SPAccenT---TNI---Hindi,0.076503,0.032392,0.7237,0.2771,50.0,50.0
AC_normal---SLT---Hindi,0.128962,0.072674,0.9435,0.4878,50.0,50.0
AC_large---SLT---Hindi,0.113661,0.059385,0.8351,0.4362,50.0,50.0
AC_normal_VCTK---SLT---Hindi,0.125683,0.056063,0.9483,0.5256,50.0,50.0
